In [1]:
import pandas as pd
import pickle
import os
import time
from task1 import get_drug_post
import concurrent.futures
import json
import openai
from pararallelize import process_posts_in_parallel

with open('../data/secrets.json') as f:
    secrets = json.load(f)

api_key = secrets['OPENAI_API_KEY_LB']

client = openai.Client(api_key=api_key)

In [2]:
dat = pd.read_excel('../data/evaluation_sets/validation_layer_eval.xlsx')

In [3]:
dat["gpt3_5_label"] = process_posts_in_parallel(dat["text"].tolist(),model= "gpt-3.5-turbo-0125",task = get_drug_post, client= client)

In [4]:
# extract records that are d
drug_posts = dat[dat["gpt3_5_label"] == "d"]

In [5]:
drug_posts["gpt4_label"] = process_posts_in_parallel(drug_posts["text"].tolist(),model= "gpt-4-turbo-2024-04-09",task = get_drug_post, client= client)
print("num of D lables: ", len(drug_posts[drug_posts["gpt4_label"] == "d"]))

TPM limit reached. Pausing for a minute...


/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_20804/2843573097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug_posts["gpt4_label"] = process_posts_in_parallel(drug_posts["text"].tolist(),model= "gpt-4-turbo-2024-04-09",task = get_drug_post, client= client)


In [6]:
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(drug_posts["is_drug_LB"], drug_posts["gpt4_label"]))

              precision    recall  f1-score   support

           d       0.94      0.91      0.92       212
          nd       0.59      0.68      0.63        40

    accuracy                           0.87       252
   macro avg       0.76      0.79      0.78       252
weighted avg       0.88      0.87      0.88       252

